In [1]:
import sys
import pathlib
import numpy as np
import pandas as pd
import dask.dataframe as dd
from tqdm import tqdm

ROOT = pathlib.Path().absolute().parent.parent
RAW_DATA_PATH = ROOT / 'data' / 'raw'
INTERIM_DATA_PATH = ROOT / 'data' / 'interim'
PROCESSED_DATA_PATH = ROOT / 'data' / 'processed'
DAYS_PRED = 28

# endure this project is in the path
sys.path.insert(0, ROOT.absolute().as_posix())
from src.data.process_data import reduce_memory_usage

from distributed import Client
# client = Client()

In [3]:
df = pd.read_parquet(PROCESSED_DATA_PATH / 'train_validation.parquet')
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46881677 entries, 0 to 46881676
Data columns (total 86 columns):
 #   Column                           Dtype         
---  ------                           -----         
 0   part                             category      
 1   id                               category      
 2   item_id                          category      
 3   dept_id                          category      
 4   cat_id                           category      
 5   store_id                         category      
 6   state_id                         category      
 7   d                                category      
 8   demand                           uint16        
 9   date                             datetime64[ns]
 10  weekday                          category      
 11  wday                             int64         
 12  month                            int64         
 13  year                             int64         
 14  event_type_cultural             

,part,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,...,quarter,weekofyear,day,dayofyear,is_year_end,is_year_start,is_quarter_end,is_quarter_start,is_month_end,is_month_start
0,train,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3,2011-01-29,...,1,4,29,29,False,False,False,False,False,False
1,train,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_2,0,2011-01-30,...,1,4,30,30,False,False,False,False,False,False
2,train,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_3,0,2011-01-31,...,1,5,31,31,False,False,False,False,True,False
3,train,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_4,1,2011-02-01,...,1,5,1,32,False,False,False,False,False,True
4,train,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_5,4,2011-02-02,...,1,5,2,33,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881672,validation,HOUSEHOLD_2_516_WI_3_validation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_1937,0,2016-05-18,...,2,20,18,139,False,False,False,False,False,False
46881673,validation,HOUSEHOLD_2_516_WI_3_validation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_1938,0,2016-05-19,...,2,20,19,140,False,False,False,False,False,False
46881674,validation,HOUSEHOLD_2_516_WI_3_validation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_1939,0,2016-05-20,...,2,20,20,141,False,False,False,False,False,False
46881675,validation,HOUSEHOLD_2_516_WI_3_validation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_1940,0,2016-05-21,...,2,20,21,142,False,False,False,False,False,False


In [45]:
dtypes = {x:'bool' for x in df.columns if x.startswith('is_')}
dtypes.update({x:'float16' for x in df.columns if x.startswith('rolling_')})
dtypes.update({x:'uint16' for x in df.columns if x.startswith('rolling_nonzero')})
dtypes.update({x:'uint8' for x in ['quarter','weekofyear','day']})
dtypes.update({x:'uint16' for x in ['dayofyear']})

df = \
(dd
.from_pandas(pd.read_pickle(PROCESSED_DATA_PATH / 'train_validation.pickle'), npartitions=100)
.persist()
.groupby('id')
.transform(lambda g: g.replace([-np.inf,np.inf],np.nan).fillna(method='bfill').fillna(method='ffill'))
.astype(dtypes)
.compute())

df

tornado.application - ERROR - Exception in callback <bound method BokehTornado._keep_alive of <bokeh.server.tornado.BokehTornado object at 0x000002154024D808>>
Traceback (most recent call last):
  File "C:\Users\ylene\Anaconda3\envs\M5-forecasting\lib\site-packages\tornado\ioloop.py", line 907, in _run
    return self.callback()
  File "C:\Users\ylene\Anaconda3\envs\M5-forecasting\lib\site-packages\bokeh\server\tornado.py", line 579, in _keep_alive
    c.send_ping()
  File "C:\Users\ylene\Anaconda3\envs\M5-forecasting\lib\site-packages\bokeh\server\connection.py", line 80, in send_ping
    self._socket.ping(codecs.encode(str(self._ping_count), "utf-8"))
  File "C:\Users\ylene\Anaconda3\envs\M5-forecasting\lib\site-packages\tornado\websocket.py", line 447, in ping
    raise WebSocketClosedError()
tornado.websocket.WebSocketClosedError


KeyboardInterrupt: 

The only columns which have negatives are:

rolling_skew_t30<br>
rolling_kurt_t30<br>
price_change_t1<br>
price_change_t365<br>

In [46]:
df

,part,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,weekday,wday,month,year,event_type_cultural,event_type_national,event_type_religious,event_type_sporting,event_name_chanukah end,event_name_christmas,event_name_cinco de mayo,event_name_columbusday,event_name_easter,event_name_eid al-fitr,event_name_eidaladha,event_name_father's day,event_name_halloween,event_name_independenceday,event_name_laborday,event_name_lentstart,event_name_lentweek2,event_name_martinlutherkingday,event_name_memorialday,event_name_mother's day,event_name_nbafinalsend,event_name_nbafinalsstart,event_name_newyear,event_name_orthodoxchristmas,event_name_orthodoxeaster,event_name_pesach end,event_name_presidentsday,event_name_purim end,event_name_ramadan starts,event_name_stpatricksday,event_name_superbowl,event_name_thanksgiving,event_name_valentinesday,event_name_veteransday,is_weekend,event,snap,sell_price_cent,shift_t0,shift_t1,shift_t2,rolling_std_t7,rolling_std_t30,rolling_std_t60,rolling_std_t90,rolling_std_t180,rolling_mean_t7,rolling_mean_t30,rolling_mean_t60,rolling_mean_t90,rolling_mean_t180,rolling_skew_t30,rolling_kurt_t30,rolling_nonzero_sale_count_t7,rolling_nonzero_sale_count_t30,rolling_nonzero_sale_count_t60,rolling_nonzero_sale_count_t90,rolling_nonzero_sale_count_t180,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t30,quarter,weekofyear,day,dayofyear,is_year_end,is_year_start,is_quarter_end,is_quarter_start,is_month_end,is_month_start
npartitions=100,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,category[known],category[known],category[known],category[known],category[known],category[known],category[known],category[known],uint16,datetime64[ns],category[known],category[known],category[known],category[known],bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,uint16,uint16,uint16,uint16,float16,float16,float16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int8,int8,int8,int32,int8,int8,int8,int8,int8,int8
468817,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46412883,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881676,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [44]:
# x = df.select_dtypes(include=['float']).isna().sum().compute()>0
# df.select_dtypes(include=['float']).columns[x]
df.select_dtypes(include=['float']).isna().sum().compute()

rolling_std_t90                     1142
rolling_std_t180                    6037
rolling_mean_t7                        0
rolling_mean_t30                       0
rolling_mean_t60                       0
rolling_mean_t90                    1142
rolling_mean_t180                   6037
rolling_skew_t30                       0
rolling_kurt_t30                       0
rolling_nonzero_sale_count_t7          0
rolling_nonzero_sale_count_t30         0
rolling_nonzero_sale_count_t60         0
rolling_nonzero_sale_count_t90      1142
rolling_nonzero_sale_count_t180     6037
price_change_t1                        0
price_change_t365                  70931
rolling_price_std_t7                   0
rolling_price_std_t30                  0
dtype: int64

In [47]:
mask = df.rolling_std_t90.isna().compute()
x = df[mask].compute()
x

,part,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,...,quarter,weekofyear,day,dayofyear,is_year_end,is_year_start,is_quarter_end,is_quarter_start,is_month_end,is_month_start
10150187,train,FOODS_3_595_CA_1_validation,FOODS_3_595,FOODS_3,FOODS,CA_1,CA,d_1842,0,2016-02-13,...,1,6,13,44,0,0,0,0,0,0
10150188,train,FOODS_3_595_CA_1_validation,FOODS_3_595,FOODS_3,FOODS,CA_1,CA,d_1843,0,2016-02-14,...,1,6,14,45,0,0,0,0,0,0
10150189,train,FOODS_3_595_CA_1_validation,FOODS_3_595,FOODS_3,FOODS,CA_1,CA,d_1844,0,2016-02-15,...,1,7,15,46,0,0,0,0,0,0
10150190,train,FOODS_3_595_CA_1_validation,FOODS_3_595,FOODS_3,FOODS,CA_1,CA,d_1845,0,2016-02-16,...,1,7,16,47,0,0,0,0,0,0
10150191,train,FOODS_3_595_CA_1_validation,FOODS_3_595,FOODS_3,FOODS,CA_1,CA,d_1846,3,2016-02-17,...,1,7,17,48,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37535708,validation,HOUSEHOLD_1_400_WI_2_validation,HOUSEHOLD_1_400,HOUSEHOLD_1,HOUSEHOLD,WI_2,WI,d_1937,0,2016-05-18,...,2,20,18,139,0,0,0,0,0,0
37535709,validation,HOUSEHOLD_1_400_WI_2_validation,HOUSEHOLD_1_400,HOUSEHOLD_1,HOUSEHOLD,WI_2,WI,d_1938,0,2016-05-19,...,2,20,19,140,0,0,0,0,0,0
37535710,validation,HOUSEHOLD_1_400_WI_2_validation,HOUSEHOLD_1_400,HOUSEHOLD_1,HOUSEHOLD,WI_2,WI,d_1939,0,2016-05-20,...,2,20,20,141,0,0,0,0,0,0
37535711,validation,HOUSEHOLD_1_400_WI_2_validation,HOUSEHOLD_1_400,HOUSEHOLD_1,HOUSEHOLD,WI_2,WI,d_1940,0,2016-05-21,...,2,20,21,142,0,0,0,0,0,0


In [72]:
df[df.id=='FOODS_3_595_CA_1_validation'].compute().select_dtypes(exclude=['bool']).iloc[:,:20]

,part,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,weekday,wday,month,year,sell_price_cent,shift_t0,shift_t1,shift_t2,rolling_std_t7,rolling_std_t30
10150187,train,FOODS_3_595_CA_1_validation,FOODS_3_595,FOODS_3,FOODS,CA_1,CA,d_1842,0,2016-02-13,Saturday,1,2,2016,198,0,0,0,1.133789,1.216797
10150188,train,FOODS_3_595_CA_1_validation,FOODS_3_595,FOODS_3,FOODS,CA_1,CA,d_1843,0,2016-02-14,Sunday,2,2,2016,198,0,0,0,1.133789,1.216797
10150189,train,FOODS_3_595_CA_1_validation,FOODS_3_595,FOODS_3,FOODS,CA_1,CA,d_1844,0,2016-02-15,Monday,3,2,2016,198,0,0,0,1.133789,1.216797
10150190,train,FOODS_3_595_CA_1_validation,FOODS_3_595,FOODS_3,FOODS,CA_1,CA,d_1845,0,2016-02-16,Tuesday,4,2,2016,198,0,0,0,1.133789,1.216797
10150191,train,FOODS_3_595_CA_1_validation,FOODS_3_595,FOODS_3,FOODS,CA_1,CA,d_1846,3,2016-02-17,Wednesday,5,2,2016,198,3,0,0,1.133789,1.216797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10150282,validation,FOODS_3_595_CA_1_validation,FOODS_3_595,FOODS_3,FOODS,CA_1,CA,d_1937,0,2016-05-18,Wednesday,5,5,2016,198,0,0,0,0.689941,1.081055
10150283,validation,FOODS_3_595_CA_1_validation,FOODS_3_595,FOODS_3,FOODS,CA_1,CA,d_1938,0,2016-05-19,Thursday,6,5,2016,198,0,0,0,0.577148,1.072266
10150284,validation,FOODS_3_595_CA_1_validation,FOODS_3_595,FOODS_3,FOODS,CA_1,CA,d_1939,0,2016-05-20,Friday,7,5,2016,198,0,0,0,0.689941,1.072266
10150285,validation,FOODS_3_595_CA_1_validation,FOODS_3_595,FOODS_3,FOODS,CA_1,CA,d_1940,0,2016-05-21,Saturday,1,5,2016,198,0,0,0,0.689941,1.072266


In [59]:
x.groupby('id', observed=True).size()#.rolling_std_t90.isna().sum()-len(x)#[x.id=='FOODS_3_595_CA_1_validation']

id
FOODS_3_595_CA_1_validation        100
FOODS_3_595_CA_3_validation        100
FOODS_3_296_CA_4_validation        114
HOUSEHOLD_1_311_CA_2_validation    100
HOUSEHOLD_1_405_CA_2_validation    100
HOUSEHOLD_1_278_CA_3_validation    100
HOUSEHOLD_1_512_CA_3_validation    114
HOUSEHOLD_1_400_CA_4_validation    100
HOUSEHOLD_1_386_WI_1_validation    100
HOUSEHOLD_1_020_WI_2_validation    100
HOUSEHOLD_1_400_WI_2_validation    114
dtype: int64

In [66]:
xx = \
(pd
.read_feather(INTERIM_DATA_PATH / 'sell_prices_reduced.feather')
.query("""item_id=='HOUSEHOLD_1_311' and store_id=='CA_2'""")
)

xx.shape[0]*7

133

In [ ]:
dtypes = {x:'bool' for x in df.columns if x.startswith('is_')}
dtypes.update({x:'float16' for x in df.columns if x.startswith('rolling_')})
dtypes.update({x:'uint16' for x in df.columns if x.startswith('rolling_nonzero')})
dtypes.update({x:'uint8' for x in ['quarter','weekofyear','day']})
dtypes.update({x:'uint16' for x in ['dayofyear']})

In [35]:
# x = df.select_dtypes(exclude=['bool','category','datetime']).min().compute()
df.select_dtypes(exclude=['bool','category','datetime']).max().compute()

demand                               763.000000
sell_price_cent                    10732.000000
shift_t0                             763.000000
shift_t1                             763.000000
shift_t2                             763.000000
rolling_std_t7                       254.125000
rolling_std_t30                      200.250000
rolling_std_t60                      187.375000
rolling_std_t90                      180.500000
rolling_std_t180                     167.500000
rolling_mean_t7                      603.000000
rolling_mean_t30                     434.500000
rolling_mean_t60                     310.250000
rolling_mean_t90                     279.750000
rolling_mean_t180                    251.000000
rolling_skew_t30                       5.477226
rolling_kurt_t30                      30.000000
rolling_nonzero_sale_count_t7          7.000000
rolling_nonzero_sale_count_t30        30.000000
rolling_nonzero_sale_count_t60        60.000000
rolling_nonzero_sale_count_t90        90

In [6]:
df.iloc[200]

part                                      train
id                  FOODS_1_001_CA_1_validation
item_id                             FOODS_1_001
dept_id                                 FOODS_1
cat_id                                    FOODS
                               ...             
is_year_start                                 0
is_quarter_end                                0
is_quarter_start                              0
is_month_end                                  0
is_month_start                                0
Name: 200, Length: 86, dtype: object

In [40]:
np.finfo(np.float16)

finfo(resolution=0.001, min=-6.55040e+04, max=6.55040e+04, dtype=float16)